# Query Wikidata for Belgian politicians

In [1]:
from datetime import datetime as dt

from SPARQLWrapper import SPARQLWrapper, JSON

## Fonctions

In [2]:
def get_rows():
    """Retrieve results from SPARQL"""
    endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
    sparql = SPARQLWrapper(endpoint)

    statement = """
    SELECT DISTINCT ?person ?personLabel ?dateBirth ?dateDeath WHERE {
        ?person wdt:P27 wd:Q31 .
        ?person wdt:P106 wd:Q82955 .
        ?person wdt:P569 ?dateBirth .
        OPTIONAL {?person wdt:P570 ?dateDeath .}
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
    }
    ORDER BY ?personLabel
    """

    sparql.setQuery(statement)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    rows = results['results']['bindings']
    print(f"\n{len(rows)} Belgian politicians found\n")
    return rows

In [3]:
def show(rows, name_filter=None, n=10):
    """Display n politicians (default=10)"""
    date_format = "%Y-%m-%dT%H:%M:%SZ"
    if name_filter:
        rows = [row for row in rows if name_filter in row['personLabel']['value'].lower()]
    print(f"Displaying the first {n}:\n")
    for row in rows[:n]:
        try:
            birth_date = dt.strptime(row['dateBirth']['value'], date_format)
            birth_year = birth_date.year
        except ValueError:
            birth_year = "????"
        try:
            death_date = dt.strptime(row['dateDeath']['value'], date_format)
            death_year = death_date.year
        except ValueError: # unknown death date
            death_year = "????"
        except KeyError: # still alive
            death_year = ""
        print(f"{row['personLabel']['value']} ({birth_year}-{death_year})")

## Obtenir une liste des politiciens belges depuis Wiki Data


In [4]:
rows = get_rows()


7489 Belgian politicians found



In [5]:
rows[:10]

[{'person': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q101444565'},
  'personLabel': {'type': 'literal', 'value': 'Q101444565'},
  'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
   'type': 'literal',
   'value': '1967-07-03T00:00:00Z'}},
 {'person': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q17350686'},
  'personLabel': {'xml:lang': 'en',
   'type': 'literal',
   'value': 'Abdallah Kanfaoui'},
  'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
   'type': 'literal',
   'value': '1968-05-21T00:00:00Z'}},
 {'person': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q13461801'},
  'personLabel': {'xml:lang': 'en',
   'type': 'literal',
   'value': 'Abdelaziz Charkaoui Hajri'},
  'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
   'type': 'literal',
   'value': '1951-01-01T00:00:00Z'}},
 {'person': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q27903935'},
  'pers

## Imprimer les 30 premiers politiciens


In [6]:
show(rows, n=30)

Displaying the first 30:

Q101444565 (1967-)
Abdallah Kanfaoui (1968-)
Abdelaziz Charkaoui Hajri (1951-)
Abderrahim Lahlali (1978-)
Abdon Comyn (1914-1965)
Abdon Demarneffe (1906-1992)
Abel Braconnier (1870-1936)
Abel Charloteaux (1897-1982)
Abel Dubois (1921-1989)
Abel de Kerchove d'Exaerde (1839-1914)
Abraham Leon (1918-1944)
Achiel De Keyser (1889-1961)
Achiel Denys (1878-1933)
Achiel Heyman (1888-1975)
Achiel Lauwers (1864-1910)
Achiel Pittery (1912-2008)
Achiel Smets (1931-2005)
Achiel Vandenabeele (1922-2018)
Achiel Verstraete (1898-1980)
Achiel Wallays (1895-1978)
Achille Albert Eeman (1852-1920)
Achille Colignon (1813-1891)
Achille Debrus (1934-)
Achille Delattre (1879-1964)
Achille Diegenant (1936-2019)
Achille Heyndrickx (1888-1953)
Achille Huart Hamoir (1841-1913)
Achille Legrand (1831-1897)
Achille Van Acker (1898-1975)
Achille Vleurinck (1913-1959)


## Filter pour afficher seulement les noms contenant "elio"


In [8]:
show(rows, name_filter="elio")

Displaying the first 10:

Auguste Criquelion (1882-1967)
Elio Di Rupo (1951-)


## Obtenir une liste des politiciens belges qui étaient également bourgmestres depuis Wiki Data 

In [9]:
def get_rows():
    """Retrieve results from SPARQL"""
    endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
    sparql = SPARQLWrapper(endpoint)

    statement = """
    SELECT DISTINCT ?person ?personLabel ?dateBirth ?dateDeath WHERE {
        ?person wdt:P27 wd:Q31 .
        ?person wdt:P21 wd:Q6581097 .
        ?person wdt:P106 wd:Q82955 .
        ?person wdt:P569 ?dateBirth .
        ?person wdt:P39 wd:Q33126365 .
        OPTIONAL {?person wdt:P570 ?dateDeath .}
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
    }
    ORDER BY ?personLabel
    """

    sparql.setQuery(statement)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    rows = results['results']['bindings']
    print(f"\n{len(rows)} Belgian politicians who also were bourgmestre found\n")
    return rows

In [10]:
def show(rows, name_filter=None, n=10):
    """Display n politicians (default=10)"""
    date_format = "%Y-%m-%dT%H:%M:%SZ"
    if name_filter:
        rows = [row for row in rows if name_filter in row['personLabel']['value'].lower()]
    print(f"Displaying the first {n}:\n")
    for row in rows[:n]:
        try:
            birth_date = dt.strptime(row['dateBirth']['value'], date_format)
            birth_year = birth_date.year
        except ValueError:
            birth_year = "????"
        try:
            death_date = dt.strptime(row['dateDeath']['value'], date_format)
            death_year = death_date.year
        except ValueError: # unknown death date
            death_year = "????"
        except KeyError: # still alive
            death_year = ""
        print(f"{row['personLabel']['value']} ({birth_year}-{death_year})")

In [11]:
rows = get_rows()


18 Belgian politicians who also were bourgmestre found



## Imprimer les 10 premiers politiciens également bourgmestre

In [12]:
show(rows, n=10)

Displaying the first 10:

Adolphe Max (1869-1939)
Charles-Joseph, 4th Duke d'Ursel (1777-1860)
Felix Vanderstraeten (1823-1884)
François-Xavier de Donnea (1941-)
Freddy Thielemans (1944-)
Guillaume Van Volxem (1791-1868)
Hervé Brouhon (1924-1993)
Jan Grauls (1887-1960)
Jan-Baptist Verlooy (1746-1797)
Jean de Locquenghien (1518-1574)
